In [11]:
# Importamos las librerías necesarias para convertir el pdf a texto
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

# Definimos la función que convierte el pdf a texto
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [12]:
# Guardamos el texto en un nuevo archivo
with open('Madrid_Parques.txt', 'w', encoding='utf-8') as f:
    text = convert_pdf_to_txt('Madrid_Parques.pdf')
    f.write(text)

In [2]:
# Leemos el texto por líneas
with open('Madrid_Parques.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

In [3]:
# Extraemos los nombres de los centros
centros = []
for i, line in enumerate(lines):
    if line.startswith('MADRID'):
        for j in range(i+2, len(lines)):
            if lines[j] == '\n':
                break
            else:
                centros.append(lines[j])

# Limpiamos y juntamos
centros = [centro.replace('\n', '').replace('  ', ' ') for centro in centros]
centros = [centros[0] + ' ' + centros[1] + ' ' + centros[2],
            centros[3] + ' ' + centros[4][:-1] + centros[5],
            centros[6] + ' ' + centros[7],
            centros[8] + ' ' + centros[9][:-2] + centros[10]]
centros = [(centro.split('.')[0], centro.split('.')[1][1:]) for centro in centros]

In [4]:
centros

[('Parque Natural de Peñalara',
  'Centro de Educación Ambiental Puente del Perdón'),
 ('Parque Regional de la Cuenca Alta del Manzanares',
  'Centro de Educación Ambiental Manzanares'),
 ('Reserva de la Biosfera de la Sierra del Rincón', 'Hayedo de Montejo'),
 ('Reserva de la Biosfera de la Sierra del Rincón',
  'Senda del Agua y Lagna del Salmoral')]

In [5]:
import pandas as pd

# Obtenemos datos de cada centro
data1 = {'Ubicacion':[], 'Centro':[], 'Direccion':[], 'Telefono':[], 'email':[],'Informacion':[]}

clean_lines = [line.replace('\n', '') for line in lines]

for centro in centros:

    data1['Ubicacion'].append(centro[0])
    data1['Centro'].append(centro[1])

    for i, line in enumerate(clean_lines):
        if line == centro[1]:
            for j in range(i+2, len(clean_lines)):
                if clean_lines[j] == 'Información General':
                    texto = ''
                    for k in range(j+2, len(clean_lines)):
                        if clean_lines[k] == '':
                            data1['Informacion'].append(texto)
                            break
                        else:
                            texto += ' ' + clean_lines[k]
                elif j == i + 2:
                    data1['Direccion'].append(clean_lines[j] + ' ' + clean_lines[j+1])
                    data1['Telefono'].append(''.join(clean_lines[j+2].replace('Teléfono / Fax: ', '').split(' ')))
                    data1['email'].append(clean_lines[j+3].replace('Email: ', ''))

centros_df = pd.DataFrame(data1, index=range(len(data1['Centro'])))

In [6]:
centros_df

,Ubicacion,Centro,Direccion,Telefono,email,Informacion
0,Parque Natural de Peñalara,Centro de Educación Ambiental Puente del Perdón,Carretera M-604 Km. 28 28740 Rascafría (Madrid),918691757,redcentrospuentedp@yahoo.es,El “Centro de Educación Ambiental Puente del ...
1,Parque Regional de la Cuenca Alta del Manzanares,Centro de Educación Ambiental Manzanares,Carretera M-604 Km. 28 28740 Rascafría (Madrid),918691757,redcentrospuentedp@yahoo.es,El Centro dispone de un edificio principal co...
2,Reserva de la Biosfera de la Sierra del Rincón,Hayedo de Montejo,Carretera M-604 Km. 28 28740 Rascafría (Madrid),918691757,redcentrospuentedp@yahoo.es,El “Hayedo de Montejo” constituye uno de los ...
3,Reserva de la Biosfera de la Sierra del Rincón,Senda del Agua y Lagna del Salmoral,Carretera M-604 Km. 28 28740 Rascafría (Madrid),918691757,redcentrospuentedp@yahoo.es,La “Senda del Agua” une el municipio de Práde...


In [116]:
# Obtenemos accesibilidad de cada centro
data2 = {'Centro':[]}

excepciones = ['Espacios y Actividades en la Naturaleza Accesibles para Todas las Personas',
            'Centros de interpretación / Casas del Parque y Aulas de la Naturaleza',
            'Ubicación',
            'Características destacables',
            'Zona de Picnic',
            '(cid:2) Dispone de paneles informati-',
            'trastado: Sí',
            'rrelieve: No',
            '(cid:2) Iconos homologados: no hay',
            'Señalización',
            'Sendero Guiado / Autoguiado',
            'pavimento',
            'Arboreto']

found_cols = [] # Columnas que se añaden por centro

In [123]:
centro[1]

'Senda del Agua y Lagna del Salmoral'

In [119]:
# Extracción de datos
for centro in centros:

    data2['Centro'].append(centro[1])
    print(len(data2['Centro']))

    for i, line in enumerate(clean_lines):
        if line == centro[1]:
            for j in range(i+2, len(clean_lines)):
                if clean_lines[j] == 'Accesibilidad Visual':
                    break
                elif clean_lines[j] == 'Accesibilidad Física':
                    for k in range(j+1, len(clean_lines)-1):
                        # Fin de los datos del centro
                        if clean_lines[k] == 'Accesibilidad Visual':
                            print(found_cols)
                            # Se mira el número de columnas y se añade 'NP' en las que existen pero no se han encontrado
                            for key in data2.keys():
                                if key not in found_cols and key != 'Centro':
                                    data2[key].append('NP')
                            found_cols = [] # Se reinicia la cuenta
                            break
                        # Columna encontrada
                        elif clean_lines[k] != '' and clean_lines[k+1] == '' and clean_lines[k-1] == '' and clean_lines[k].replace(' ', '').isalpha() and '' not in clean_lines[k] and clean_lines[k] not in excepciones:
                            found_cols.append(clean_lines[k])
                            texto = ''
                            # Concatenando líneas de la respectiva columna
                            for n in range(k+2, len(clean_lines)):
                                #print(len(data2['Centro']))
                                if clean_lines[n] == '':
                                    if clean_lines[k] in data2.keys():
                                        data2[clean_lines[k]].append(texto.replace('- ', ''))
                                        break
                                    elif len(data2['Centro']) == 1:
                                        data2[clean_lines[k]] = [texto.replace('- ', '')]
                                        break
                                    else:
                                        data2[clean_lines[k]] = ['NP' for v in range(len(data2['Centro']))].append(texto.replace('- ', ''))
                                        break
                                # Nos saltamos los cambios de página.
                                elif clean_lines[n] == '' and len(clean_lines[n+1]) == 1 or len(clean_lines[n]) == 1 and len(clean_lines[n+1]) == 1:
                                    continue
                                else:
                                    texto += clean_lines[n] + ' '


access_df = pd.DataFrame(data2, index=range(len(data2['Centro'])))

1
['Acceso ', 'Recepción', 'Aseo adaptado', 'Inodoro', 'Lavabo y accesorios del aseo', 'Senderos', 'Mobiliario y zonas de descanso']
2
3
4


In [99]:
c1 = pd.read_csv('centros_madrid.csv')
p1 = pd.read_csv('parques_madrid.csv')

In [103]:
c1

,Unnamed: 0,Ubicacion,Centro,Direccion,Telefono,email,Informacion
0,0,Parque Natural de Peñalara,Centro de Educación Ambiental Puente del Perdón,Carretera M-604 Km. 28 28740 Rascafría (Madrid),918691757,redcentrospuentedp@yahoo.es,El “Centro de Educación Ambiental Puente del ...
1,1,Parque Regional de la Cuenca Alta del Manzanares,Centro de Educación Ambiental Manzanares,Carretera M-604 Km. 28 28740 Rascafría (Madrid),918691757,redcentrospuentedp@yahoo.es,El Centro dispone de un edificio principal co...
2,2,Reserva de la Biosfera de la Sierra del Rincón,Hayedo de Montejo,Carretera M-604 Km. 28 28740 Rascafría (Madrid),918691757,redcentrospuentedp@yahoo.es,El “Hayedo de Montejo” constituye uno de los ...
3,3,Reserva de la Biosfera de la Sierra del Rincón,Senda del Agua y Lagna del Salmoral,Carretera M-604 Km. 28 28740 Rascafría (Madrid),918691757,redcentrospuentedp@yahoo.es,La “Senda del Agua” une el municipio de Práde...


In [124]:
p1

,Unnamed: 0,Centro,Acceso,Recepción,Aseo adaptado,Inodoro,Lavabo y accesorios del aseo,Senderos,Mobiliario y zonas de descanso
0,0,Centro de Educación Ambiental Puente del Perdón,Para acceder desde el aparcamiento hasta el ed...,Se accede desde las áreas temáticas al aire li...,"Hay una cabina de aseo adaptada, independiente...",El inodoro tiene un espacio libre de acercamie...,El lavabo permite la aproximación frontal de u...,Las sendas que recorren el “Área de Recursos A...,Dispone de una zona de descanso al aire libre ...
1,1,Centro de Educación Ambiental Manzanares,Para acceder desde el aparcamiento hasta el ed...,Se accede desde las áreas temáticas al aire li...,"Hay una cabina de aseo adaptada, independiente...",El inodoro tiene un espacio libre de acercamie...,El lavabo permite la aproximación frontal de u...,Las sendas que recorren el “Área de Recursos A...,Dispone de una zona de descanso al aire libre ...
2,2,Hayedo de Montejo,Para acceder desde el aparcamiento hasta el ed...,Se accede desde las áreas temáticas al aire li...,"Hay una cabina de aseo adaptada, independiente...",El inodoro tiene un espacio libre de acercamie...,El lavabo permite la aproximación frontal de u...,Las sendas que recorren el “Área de Recursos A...,Dispone de una zona de descanso al aire libre ...
3,3,Senda del Agua y Lagna del Salmoral,Para acceder desde el aparcamiento hasta el ed...,Se accede desde las áreas temáticas al aire li...,"Hay una cabina de aseo adaptada, independiente...",El inodoro tiene un espacio libre de acercamie...,El lavabo permite la aproximación frontal de u...,Las sendas que recorren el “Área de Recursos A...,Dispone de una zona de descanso al aire libre ...
